In [46]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
#import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
import operator

In [47]:
val_all_path = '../input/cmncthesis/C-NMC_test_final_phase_data'

In [48]:
val_all_list = os.listdir(val_all_path)
val_all_list.sort()

In [49]:
print('validation: ', len(val_all_list))

validation:  2586


In [50]:
val_all_batch = np.zeros((len(val_all_list), 150, 150, 3), dtype=np.uint8)

In [51]:
print(val_all_batch.shape)

(2586, 150, 150, 3)


In [52]:
def Read_n_Crop(list_data, batch, path):
    i=0
    for x in list_data:
        image = cv2.imread(os.path.join(path, x))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = crop_center(image, (150,150,3))
        batch[i] = image
        i+=1
    
    print(type(batch), batch.shape, batch.dtype, batch[0].shape, batch[0].dtype)
    return batch

In [53]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [54]:
images=Read_n_Crop(val_all_list, val_all_batch, val_all_path)
images = images/255.0

<class 'numpy.ndarray'> (2586, 150, 150, 3) uint8 (150, 150, 3) uint8


In [55]:
model= None
# load the trained model
model = load_model('../input/effnetb0-5xaug-test8/EffnetB0_5xAug_test8.h5', compile=False)

In [56]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 150, 150, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [57]:
predictions = model.predict(images, verbose = 10)

In [58]:
y_pred_flat = []
for pred in predictions:
   if pred > 0.3:
        y_pred_flat.append(0)
   else:
       y_pred_flat.append(1)
y_pred_flat = np.array(y_pred_flat)

In [59]:
print(len(y_pred_flat))

2586


In [60]:
with open('isbi_valid.predict', 'w') as f:
    for item in y_pred_flat:
        f.write("%s\n" %item)